In [1]:
import urllib
import sys
import basics
import json
import geopandas as gpd
import pandas as pd
import numpy as np
import string
from urllib.parse import quote
from imp import reload
reload(basics)

<module 'basics' from 'F:\\study1\\研一\\公交出行数据采集\\basics.py'>

In [2]:
# 采集出行路径空间数据
def GetTotalLine(ak,opoint,dpoint,date,time,city,filename):
    requesturl='https://restapi.amap.com/v3/direction/transit/integrated?origin='+opoint.lon+','+opoint.lat\
                +"&destination="+dpoint.lon+','+dpoint.lat+"&city="+city+"&strategy=0&nightflag=0&key="+ak
    print(requesturl)
#     requesturl = quote(requesturl, safe=string.printable)
#     response=urllib.request.urlopen(requesturl)
#     json_obj = response.read().decode('utf-8','ignore').replace(u'\xa9', u'')
    json_obj=urllib.request.urlopen(requesturl)
    mydata=json.load(json_obj)
    if (mydata['info'] == 'OK'):
        routes=mydata['route']
        transits=routes['transits']
        if (transits != []):
            project1=transits[0]
            cost=project1['cost']
            if cost == []:
                cost = '0'
            duration=project1['duration']
            distance=project1['distance']
            segments=project1['segments']
            route_locations=[]
            for segment in segments:
                walking = segment['walking']
                bus=segment['bus']['buslines']
                if walking != []:
                    walksteps=walking['steps']
                    for step in walksteps:
                        polylinewalk=step['polyline']
                        for loc in polylinewalk.split(';'):
                            route_locations.append(loc)
                if bus != []:
                    polylinebus=bus[0]['polyline']
                    for loc in polylinebus.split(';'):
                        route_locations.append(loc)
            usefuldata=[opoint.id,opoint.name,dpoint.id,dpoint.name,cost,duration,distance]
            with open(file = filename,mode = "a",encoding='utf-8') as doc:
                for data in usefuldata:
                    doc.write(data)
                    doc.write(';')
                for loc in route_locations:
                    doc.write(loc)
                    doc.write(';')
                doc.write('\n')
                print(i,j)
        else:
            print("距离太近，无需换乘")
            cost='0'
            distance=routes['distance']
            if (distance == []):
                distance=0
            else:
                duration=distance
                usefuldata=[opoint.id,opoint.name,dpoint.id,dpoint.name,cost,duration,distance]
                with open(file = filename,mode = "a",encoding='utf-8') as doc:
                    for data in usefuldata:
                        doc.write(data)
                        doc.write(';')
                        doc.write(opoint.lon+','+opoint.lat)
                        doc.write(';')
                        doc.write(dpoint.lon+','+dpoint.lat)
                        doc.write(';')
                        doc.write('\n')
    else:
        print("url存在问题")
        print(requesturl)
        print(mydata['info'])

In [3]:
# 采集公交出行属性数据:时间、费用、距离、换乘时间、换乘次数、步行时间等
def GetBusInformation(ak,opoint,dpoint,date,time,city,outputfile):
    requesturl='https://restapi.amap.com/v3/direction/transit/integrated?origin='+str(opoint.lon)+','+str(opoint.lat)+"&destination="+str(dpoint.lon)+','+str(dpoint.lat)+"&city="+str(city)+"&strategy=0&nightflag=0&key="+ak
#     print(requesturl)
#     requesturl = quote(requesturl, safe=string.printable)
#     response=urllib.request.urlopen(requesturl)
#     json_obj = response.read().decode('utf-8','ignore').replace(u'\xa9', u'')
    json_obj=urllib.request.urlopen(requesturl)
    mydata=json.load(json_obj)
    if mydata['info'] == 'OK':
        routes=mydata['route']
        transits=routes['transits']
        if transits != []:
            project1=transits[0]
            cost=project1['cost']
            if cost == []:
                cost='0'
            duration=project1['duration']
            distance=project1['distance']
            walking_distance=project1['walking_distance']
            segments=project1['segments']
            lensegments=len(segments)
            transits_num=lensegments-1
            railtime=0
            # 如果只坐一次车，最后没有步行环节
            if (lensegments == 1):
                first_walktime=float(segments[0]['walking']['duration'])
                last_walktime=0
                transit_walktime=0
                bustime=float(segments[0]['bus']['buslines'][0]['duration'])
                try:
                    railtime=float(segments[0]['railway']['time'])
                except Exception as e:
                    railtime=0
            elif lensegments ==2:
                first_walktime=float(segments[0]['walking']['duration'])
                try:
                    last_walktime=float(segments[lensegments-1]['walking'['duration']])
                except Exception as e:
                    last_walktime=0
                transit_walktime=0
                bustime=float(segments[0]['bus']['buslines'][0]['duration'])
                try:
                    railtime=float(segments[0]['railway']['time'])
                except Exception as e:
                    railtime=0
            elif lensegments>2: # 出发地-步行-公交-步行-换乘公交-步行-目的地
                # 起始的步行时间
                try :
                    first_walktime=float(segments[0]['walking']['duration'])
                except Exception as e:
                    first_walktime = 0
                # 最后的步行时间
                try:
                    last_walktime=float(segments[lensegments-1]['walking']['duration'])
                except Exception as e:
                    last_walktime=0
                # 初始化公交车内时间和铁路时间
                transit_walktime=0
                bustime=0
                railtime=0
                # 循环累加公交时间、换乘时间、铁路名称
                for i in range(0,lensegments):
                    if (i>=1 and i<=lensegments-2):
                        try:
                            transit_walktime+=float(segments[i]['walking']['duration'])
                        except Exception as e:
                            # 地铁换乘无需时间
                            transit_walktime+=0
                        try:
                            bustime+=float(segments[i]['bus']['buslines'][0]['duration'])
                        except Exception as e:
                            bustime+=0
                        try:#跨区域出行，该处是高铁、铁路出行时间
                            railtime+=float(segments[i]['railway']['time'])
                        except Exception as e:
                            railtime += 0
            usefuldata=[opoint.id,opoint.name,opoint.lon,opoint.lat,dpoint.id,dpoint.name,dpoint.lon,dpoint.lat,cost,duration,
                        bustime,first_walktime,last_walktime,transit_walktime,distance,walking_distance,str(transits_num),railtime]
            with open(file = outputfile,mode = "a",encoding='utf-8') as doc:
                    for data in usefuldata:
                        doc.write(str(data))
                        doc.write(';')
                    doc.write('\n')
        else:
            cost='0'
            distance=routes['distance']
            transits_num='0'
            walking_distance=distance
            if distance == []:
                distance=0
            duration=distance
            bustime=0
            first_walktime=duration
            last_walktime=0
            transit_walktime=0
            railtime=0
            usefuldata=[opoint.id,opoint.name,opint.lon,opoint.lat,dpoint.id,dpoint.name,dpoint.lon,dpoint.lat,cost,duration,
                        bustime,first_walktime,last_walktime,transit_walktime,distance,walking_distance,str(transits_num),railtime]
            with open(file = outputfile,mode = "a",encoding='utf-8') as doc:
                    for data in usefuldata:
                        doc.write(str(data))
                        doc.write(';')
                    doc.write('\n')
        print(requesturl)
    else:
        print('url存在问题')
        print(requesturl)
        print(mydata['info'])

In [7]:
# aim,ak,date,time,city,ofile,dfile,outputfile需要更改
# 参数分别为存储路径、ID、经度、纬度、名称
aim='route'
ak='your ak'
date='2020-05-25'
time='07:00'
city='330100' # 杭州市
ofile='origin.txt'
dfile='destination.txt'
outputpath='.\\data\\' # data的存放路径
outputpath1='.\\route\\' #route的存放路径
opointlist=basics.creatpoint(ofile,0,4,5,2,2)
dpointlist=basics.creatpoint(dfile,0,4,5,2,2)
for i in range(len(opointlist)):
    opoint=opointlist[i]
    if (aim=="data"):
        busoutputfile=outputpath+"bus_"+str(i)+".txt"
        doc=open(busoutputfile,'w')
        doc.close()
    elif aim=='route':
        routeoutputfile=outputpath1+"line_"+str(i)+".txt"
        doc=open(routeoutputfile,'w')
        doc.close()
    for j in range(len(dpointlist)):
        dpoint=dpointlist[j]
        if (aim=="data"):
            GetBusInformation(ak,opoint,dpoint,date,time,city,busoutputfile)
        elif (aim=="route"):
            GetTotalLine(ak,opoint,dpoint,date,time,city,routeoutputfile)

https://restapi.amap.com/v3/direction/transit/integrated?origin=120.18615,30.251941&destination=120.0875187,30.30146427&city=330100&strategy=0&nightflag=0&key=146e2bfd2ab038f962a212a71a923c0e
0 0
https://restapi.amap.com/v3/direction/transit/integrated?origin=120.139608,30.199077&destination=120.0875187,30.30146427&city=330100&strategy=0&nightflag=0&key=146e2bfd2ab038f962a212a71a923c0e
1 0
